In [2]:
import numpy as np
import tensorflow as tf

# # 从课程镜像服务器下载 MNIST NumPy 数据
# DATA_URL = "https://cdn.aibydoing.com/hands-on-ai/files/mnist.npz"

# path = tf.keras.utils.get_file("mnist.npz", DATA_URL)
# with np.load(path) as data:
#     # 将 28x28 图像 Padding 至 32x32
#     x_train = np.pad(
#         data["x_train"].reshape([-1, 28, 28, 1]),
#         ((0, 0), (2, 2), (2, 2), (0, 0)),
#         "constant",
#     )
#     y_train = data["y_train"]
#     x_test = np.pad(
#         data["x_test"].reshape([-1, 28, 28, 1]),
#         ((0, 0), (2, 2), (2, 2), (0, 0)),
#         "constant",
#     )
#     y_test = data["y_test"]

# x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [3]:
import tensorflow as tf

# 使用 keras 建立层，然后通过 estimator 基于建立好的层进行训练？
def lenet_fn(features, labels, mode):
    # 卷积层，6 个 5x5 卷积核，步长为 1，relu 激活，第一层需指定 input_shape
    conv1 = tf.keras.layers.Conv2D(filters=6,
                                   kernel_size=(5, 5),
                                   strides=(1, 1),
                                   activation='relu', input_shape=(32, 32, 1))(features["x"])
    # 平均池化，池化窗口默认为 2
    pool1 = tf.keras.layers.AveragePooling2D(
        pool_size=(2, 2), strides=2)(conv1)
    # 卷积层，16 个 5x5 卷积核，步为 1，relu 激活
    conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(
        5, 5), strides=(1, 1), activation='relu')(pool1)
    # 平均池化，池化窗口默认为 2
    pool2 = tf.keras.layers.AveragePooling2D(
        pool_size=(2, 2), strides=2)(conv2)
    # 需展平后才能与全连接层相连
    flatten = tf.keras.layers.Flatten()(pool2)
    # 全连接层，输出为 120，relu 激活
    fc1 = tf.keras.layers.Dense(units=120, activation='relu')(flatten)
    # 全连接层，输出为 84，relu 激活
    fc2 = tf.keras.layers.Dense(units=84, activation='relu')(fc1)
    # 全连接层，输出为 10，Softmax 激活
    logits = tf.keras.layers.Dense(units=10, activation='softmax')(fc2)

    # 计算损失
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # 训练模式
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # 评估模式
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels, predictions=tf.argmax(input=logits, axis=1))}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)